# SuperalloyDigger configuration file(dictionary.ini) instruction

The rule in the configuration file(dictionary.ini) file plays an important role in Superalloydigger tool. When you need to use the tool for other field, all you need to do is to modify the rule in the configuration file based on your own domain knowledge. The rule directly affects the accuracy of information extracted by the tool, so rules are best made by experts in corresponding materials fields. Different rules may work in different parts of the data extraction pipeline. Here we provided the user with an example sentence for understanding the content of each rule in the dictionary.ini and show you how to modify it in detail. 

In order to illustrate them more clearly, we divided all rules you may modify into four parts, including "1. Preprocessing rule for text", "2. Entity recognition rule for text", "3. Preprocessing rule for table", and "4. Entity recogniton rule for table". Two of the the rules act on text and the other two act on table. All the rules are used for corpus preprocessing and named entity recognition respectively.

Here we provide some example sentences to illustrate and demonstrate how the rules in dictionary.ini works in the tool.

## 1. Preprocessing rule for text

### 1.1 paras_to_replace

The rule is used for processing special phrases (contains parameter and unit) in sentences and the format of this rule can be represented as {'A':['B','C']}. Any phrase matched by rule A will be viewed as a processing object. In these processing objects, the sub-phrase matched by rule B will be replaced by string C.
Attention: A and B need to be rules, and C need to be a string.

In [3]:
import re
from dictionary import Dictionary

dict_info = Dictionary("dictionary.ini")

In [4]:
paras_to_replace = dict_info.paras_to_replace 
paras_to_replace 

{'\\s[0-9]+\\s+C': ['\\s+C', '°C'],
 '[0-9]+\\s+°C': ['\\s+', ''],
 '[0-9]+\\s+K': ['\\s+K', 'K'],
 '\\s+±\\s+': ['\\s+', ''],
 '\\s+–\\s+': ['\\s+', ''],
 '[0-9]+\\s+g/cm3': ['\\s+g/cm3', 'g/cm3'],
 '\\d\\s+%': ['\\s+%', '%'],
 '\\d\\s?K\\s?\\(\\d+\\s?\\°C\\)': ['\\(\\d+\\s?\\°C\\)', ''],
 '\\d\\s?°C\\s?\\(\\d+\\s?K\\)': ['\\(\\d+\\s?K\\)', '']}

In [5]:
sentence = "The γ′ solvus temperature of NR6 alloy is around 1175 °C, the mean grain size measured after this supersolvus solution treatment is about 26 μm."

In [7]:
for para_model, change_place in paras_to_replace.items():
    paras_all = re.findall(para_model, sentence)
    for para in paras_all:
        find_word = re.findall(change_place[0], para)
        para_out = para.replace(find_word[0], change_place[1])
        sentence = sentence.replace(para, para_out)
sentence

'The γ′ solvus temperature of NR6 alloy is around 1175°C, the mean grain size measured after this supersolvus solution treatment is about 26 μm.'

As you can see, the blank between "1175" and "°C" is removed. Since the subsequent named entity recognition is performed on the result of sentence tokenization, the preprocessing step can help recognize phrase entities in sentences.

### 1.2 replace_word

The format of this rule can be represented as {'A':'B'}. Any string A in the sentence will be replaced by string B.
Attention: A and B need to be strings.

In [8]:
from dictionary import Dictionary

dict_info = Dictionary("dictionary.ini")
replace_word = dict_info.replace_word
replace_word

{'Fig .': 'fig ',
 'Fig.': 'fig',
 'fig.': 'fig',
 'et al.': 'et al',
 '℃': '°C',
 '˚C': '°C',
 'g cm−3': 'g/cm3',
 'gm/cc': 'g/cm3',
 'g cm3': 'g/cm3',
 '–': '-',
 'L12': 'γ′',
 'incipient melting': 'incipient-melting',
 "γ'": 'γ′',
 'gamma prime': 'γ′',
 '™': 'TM',
 '<': '＜',
 '>': '＞',
 '＞ ': '＞',
 ' ＜': '＜',
 'solution heat': 'solution-heat',
 'Co3 ( Al , W )': 'γ′',
 'h.': 'h .',
 'Co3(Al,W)': 'γ′'}

In [9]:
sentence = "However, the higher relative density observed in the alloy sintered at 1200 °C, is attributed to the good densification of the alloy; this is close to the theoretical density of 7.894 g cm−3."

In [10]:
for old_word, new_word in replace_word.items():
    sentence = sentence.replace(old_word, new_word)
sentence

'However, the higher relative density observed in the alloy sintered at 1200 °C, is attributed to the good densification of the alloy; this is close to the theoretical density of 7.894 g/cm3.'

As you can see, the string "g cm−3" in the sentence has been replaced by "g/cm3". Since the subsequent named entity recognition is performed on the result of sentence tokenization, the preprocessing step can help recognize phrase entities in sentences.

### 1.3 alloy_to_replace

The rule is used for processing special phrases (contains alloy name or designation) in sentence and the format of this rule can be represented as {'A':['B','C']}. Any phrase matched by rule A will be viewed as a processing object. In these processing objects, the sub-phrase matched by rule B will be replaced by string C.
Attention: A and B need to be rules, and C needs to be a string.

In [11]:
from dictionary import Dictionary

dict_info = Dictionary("dictionary.ini")
alloy_to_replace = dict_info.alloy_to_replace
alloy_to_replace

{'\\s([A-Z]+[a-z]*\\s+[A-Z]*[0-9]{2,}[A-Z]*)\\s': ['\\s+', '~'],
 '\\s([A-Z][a-z]?[0-9]{0,2}\\.?[0-9]{0,2}\\s+[A-Z][a-z]?\\S*)\\s': ['\\s+',
  ''],
 '\\s([A-Z]+[a-z]{1,}\\s+[A-Z]+\\-?[0-9]+)\\s': ['\\s+', '~'],
 '\\s([A-Z]+[a-z]{1,}\\s+\\d+\\-?\\d*[A-Za-z]+)\\s': ['\\s+', '~'],
 '\\s([0-9]+\\.?[0-9]*\\s+[A-Z][a-z]?)\\s': ['\\s+', '']}

In [12]:
sentence = "Moreover , the computed γ′ solvus of TMW-4M3 and Alloy 720Li is 1194 °C and 1163 °C ."

In [13]:
for alloy_model, replace in alloy_to_replace.items():
    alloy_part = re.findall(alloy_model, sentence)
    for alloy in alloy_part:
        find_part = re.findall(replace[0], alloy)
        alloy_out = alloy.replace(find_part[0], replace[1])
        sentence = sentence.replace(alloy, alloy_out)
sentence

'Moreover , the computed γ′ solvus of TMW-4M3 and Alloy~720Li is 1194 °C and 1163 °C .'

As you can see, the string "Alloy 720Li" in the sentence has been replaced by "Alloy~720Li". Since the subsequent named entity recognition is performed on the result of sentence tokenization, the preprocessing step can help recognize phrase entities in sentences.

### 1.4 no_unit_para and unit_replace

no_unit_para rule and unit_replace rule are used for adding unit to the after part of parameters without a unit in the sentence.The format of no_unit_para rule can be represented as {'property_name':['A','B',..]}, and the format of unit_replace can be represented as {'property_name':['A','B',...]}. As for the no_unit_para rule, any word matched by rule A/rule B... will be viewed as no unit parameters. Then the unit strings of A, B... will be checked one by one to see if they exist in the sentence. During the check process, once a unit string is found in a sentence, the process break and add this unit string to the after part of no unit parameters.

In [14]:
from dictionary import Dictionary
import nltk

dict_info = Dictionary("dictionary.ini")
no_unit_para = dict_info.no_unit_para
unit_replace = dict_info.unit_replace
prop_name = 'solvus'

In [15]:
no_unit_para

{'solvus': ['^\\W{0,1}[7-9][0-9]\\S*\\d$', '^\\W{0,1}1[0-6][0-9]\\S*\\d$'],
 'solidus': ['^\\W{0,1}[7-9][0-9]\\S*\\d$', '^\\W{0,1}1[0-6][0-9]\\S*\\d$'],
 'density': ['^\\W?[4-9]\\.[0-9]{1,2}(\\S\\d\\.\\d{2})?$|^\\W?1[0,1]\\.[0-9]{1,2}(\\S\\d\\.\\d{2})?$'],
 'liquidus': ['^\\W{0,1}[7-9][0-9]\\S*\\d$', '^\\W{0,1}1[0-6][0-9]\\S*\\d$']}

In [16]:
unit_replace

{'solidus': ['°C', 'K'],
 'solvus': ['°C', 'K'],
 'density': ['g/cm3'],
 'liquidus': ['°C', 'K']}

In [17]:
sentence = "Moreover , the computed γ′ solvus of TMW-4M3 and Alloy 720Li is 1194 and 1163°C ."

In [18]:
para = no_unit_para[prop_name]
word_list = nltk.word_tokenize(sentence)
list_order = list(set(word_list))
for word in list_order:
    for pattern in para:
        number = re.findall(pattern, word)
        if unit_replace[prop_name]:
            if unit_replace[prop_name][0] in sentence and number:
                sent_c = sentence.replace(word, word + unit_replace[prop_name][0])
                sentence = sentence.replace(sentence, sent_c)
            if unit_replace[prop_name][0] not in sentence and number and len(unit_replace[prop_name])==2:
                sent_c = sentence.replace(word, word + unit_replace[prop_name][-1])
                sentence = sentence.replace(sentence, sent_c)
sentence

'Moreover , the computed γ′ solvus of TMW-4M3 and Alloy 720Li is 1194°C and 1163°C .'

As you can see, the string "1194" in the sentence has been replaced with "1194°C". Since the subsequent named entity recognition is performed on the result of sentence tokenization, this preprocessing step can add important unit information to it.

## 2. Entity recognition rule for text

### 2.1 alloy_writing_type

The rule is used for recognizing alloy names in the sentence. Any word matched by any rule in alloy_writing_type will be viewed as an alloy name, and participate in the relation extraction part.

In [19]:
from dictionary import Dictionary
import nltk
import re

dict_info = Dictionary("dictionary.ini")
alloy_writing_type = dict_info.alloy_writing_type
alloy_writing_type

['^[0-9]+\\.?[0-9]{0,2}[A-JL-Z]',
 '[0-9]{0,2}\\.?[0-9]{0,2}[A-Z][a-z]?\\-[0-9]{0,2}\\.?[0-9]{0,2}[A-Z][a-z]?',
 '^[A-Z]+[a-z]*\\-[0-9]\\w*',
 '^[A-Z]\\S+[0-9]$',
 '^[A-Z]+[0-9]+[A-z]+',
 '^[A-Z]+[a-z]*\\~[A-Z0-9]+']

In [20]:
sentence = "Out of these series of alloys, the alloy with a composition Co-30Ni-10Al-5Mo-2Ta-2Ti-5Cr has a solvus temperature of 1378K and a yield strength greater than 560 MPa at 1143K."

In [21]:
word_list = nltk.word_tokenize(sentence)
len_type = len(alloy_writing_type)
alloy_name = list()
for word in word_list:
    for i in range(0, len_type):
        check = re.findall(alloy_writing_type[i], word)
        if check and word not in alloy_name:
            print(alloy_writing_type[i])
            alloy_name.append(word)
alloy_name

[0-9]{0,2}\.?[0-9]{0,2}[A-Z][a-z]?\-[0-9]{0,2}\.?[0-9]{0,2}[A-Z][a-z]?


['Co-30Ni-10Al-5Mo-2Ta-2Ti-5Cr']

As you can see, the alloy name "Co-30Ni-10Al-5Mo-2Ta-2Ti-5Cr" can be recognized by our rule "[0-9]{0,2}\.?[0-9]{0,2}[A-Z][a-z]?\-[0-9]{0,2}\.?[0-9]{0,2}[A-Z][a-z]?".

## 2.2 alloy_blank_type

The rule is used for recognizing the alloy name doesn't have special writing feature but special context keywords in a sentence. Any string in the sentence matched by any rule in alloy_blank_type will be viewed as alloy name and participates in the relation extraction part.

In [22]:
from dictionary import Dictionary
import nltk
import re

dict_info = Dictionary("dictionary.ini")
alloy_blank_type = dict_info.alloy_blank_type
alloy_blank_type

['\\s([0-9A-Z]+\\w*)\\s+\\S*[Aa]lloy\\s',
 '\\s+\\S*[Aa]lloy\\s+([0-9A-Z]+\\w*)\\s']

In the above rule, "A/alloy" is used as the feature context.

In [23]:
sentence = "The mass density of the CVT alloy was measured as 8.86 g/cm3, and decreases to 8.71 g/cm3 and 8.67 g/cm3 through the addition of Ti and Al, respectively."

In [24]:
alloy_name = list()
for pattern in alloy_blank_type:
    outcome = re.findall(pattern, sentence)
    if outcome:
        print(pattern)
        for name in outcome:
            alloy_name.append(name)
alloy_name

\s([0-9A-Z]+\w*)\s+\S*[Aa]lloy\s


['CVT']

As you can see, the alloy name "CVT" can be recognized by our rule "\s([0-9A-Z]+\w*)\s+\S*[Aa]lloy\s".

## 2.3 prop_writing_type

It is a dictionary that contains writing formats of the property name, and all writing formats are top-N words with the highest similarity in a Word2vec model. The dictionary is used for recognizing the propery name in sentences and replacing all property names with a uniform format.

In [25]:
from dictionary import Dictionary
import nltk
import re

dict_info = Dictionary("dictionary.ini")
prop_writing_type = dict_info.prop_writing_type
prop_writing_type

{'solidus': ['Tsolidus', 'Solidus', 'solidus'],
 'solvus': ['γ′-solvus',
  'γ′Solvus',
  'solvus',
  'Solvus',
  'γ′-transus',
  'Tγ′',
  'solvus-temperature',
  '-solvus'],
 'density': ['density', 'Density', 'densities', 'densityρ', 'pO-'],
 'liquidus': ['liquidus', 'Liquidus', 'Tlq', 'liquidous', 'liquidusus']}

In [26]:
sentence = "The γ′-solvus in Inconel-713LC is reported to be 1205 °C which is comparable to the solidus ( 1260 °C ) of the alloy [ 4 ]."

In [27]:
prop_name = 'solvus'
for element in prop_writing_type[prop_name]:
    sentence = sentence.replace(element, prop_name)
sentence

'The solvus in Inconel-713LC is reported to be 1205 °C which is comparable to the solidus ( 1260 °C ) of the alloy [ 4 ].'

As you can see, the property name "γ′-solvus" is replaced with uniform "solvus".

## 2.4 value_wt

The rule is used for recognizing parameters and units in the sentence. Any word matched by any rule in value_wt will be viewed as a parameter with the unit, and participate in the relation extraction part.

In [28]:
from dictionary import Dictionary
import nltk
import re

dict_info = Dictionary("dictionary.ini")
value_wt = dict_info.value_wt
prop_name = "solvus"
value_wt[prop_name]

['^\\W{0,1}[7-9][0-9]{2}(\\.[0-9]{1,2})?\\S*°C$',
 '^\\W{0,1}[7-9][0-9]{2}(\\.[0-9]{1,2})?\\S*K$',
 '^\\W{0,1}1[0-6][0-9]{2}\\S*°C$',
 '^\\W{0,1}1[0-6][0-9]{2}\\S*K$']

In [29]:
sentence = "It is shown that the solvus temperature of the γ′ phase in the 0Mo alloy is near 1091°C ; however , the solvus temperature of the γ′ phase decreases with the addition of Mo : 1065°C for the 2Mo alloy and 1043°C for the 4Mo alloy ."

In [30]:
parameter_list = []
word_list = nltk.word_tokenize(sentence)
for word in word_list:
    for t in range(0, len(value_wt[prop_name])):
        check = re.findall(value_wt[prop_name][t], word)
        if check:
            parameter_list.append(word)
parameter_list

['1091°C', '1065°C', '1043°C']

As you can see, '1091°C', '1065°C', and  '1043°C' are recognized as parameters with the unit.

## 2.5 other_quality

It is a dictionary containing some property names in addition to the target property name. These property name needs to have a similar range of number to the target property and may appear in the same sentence as the target property. This dictionary is compiled by the expert for the corresponding condition of the corpus.

In [31]:
from dictionary import Dictionary
import nltk
import re

dict_info = Dictionary("dictionary.ini")
other_quality = dict_info.other_quality
prop_name = 'solvus'
other_quality[prop_name]

['γ′-solution',
 'solution',
 'solutioning',
 'solidus',
 'Tsolidus',
 'Solidus',
 'liquidus',
 'Liquidus',
 'Tlq',
 'liquidous',
 'liquidusus',
 'incipient',
 'age',
 'aged',
 'aging',
 'solution-heat',
 'ageing',
 'homogenized',
 'homogenization',
 'heat-treated',
 'super-solvus',
 'sub-solvus',
 'γ″',
 'solidification',
 'eutectic',
 'solubility',
 'annealing',
 'annealed',
 'anneal',
 'deformed',
 'melting',
 'incipient-melting',
 'supersolvus',
 'heat',
 'heated']

## 3. Preprocessing rule for table

### 3.1 table_alloy_to_replace

The format and usage of the rule are the same as alloy_to_replace (1.3). But the preprocessing object of the rule is the topic of the table. This preprocessing rule helps the recognition of alloy names in the topic.

### 3.2 ele_to_abr

The rule is used for preprocessing the element name in the table and facilitating subsequent recognition of the element name in the table. The format and usage of the rule are the same as replace_word (1.2). But the preprocessing object of the rule is the table cell. This preprocessing rule helps the subsequent position of alloy element row in the table title. The following rules can be used as a reference.

In [32]:
from dictionary import Dictionary
import nltk
import re

dict_info = Dictionary("dictionary.ini")
ele_to_abr = dict_info.ele_to_abr
ele_to_abr

{'Chromium': 'Cr',
 'Nickel': 'Ni',
 'Molybdenum': 'Mo',
 'Vanadium': 'V',
 'Aluminum': 'Al',
 'Titanium': 'Ti',
 'Boron': 'B',
 'Carbon': 'C',
 'Iron': 'Fe',
 'Manganese': 'Mg',
 'Silicon': 'Si',
 'Phosphorus': 'P',
 'Sulfur': 'S',
 'Tungsten': 'W',
 'Germanium': 'Ge',
 'Rhenium': 'Re',
 'Niobium': 'Nb',
 'Pickaxe': 'Zr',
 'Hafnium': 'Hf',
 'Ruthenium': 'Ru',
 'Iridium': 'Ir',
 'Yttrium': 'Y',
 'Platinum': 'Pt'}

## 4. Entity recogniton rule for table

### 4.1 table_prop_pattern

The rule is used for recognizing the property name in the table. The format and usage of the rule are the same as prop_writing_type (2.3). But the object of this rule is the table cell. This recognizing rule helps to position the property name column in the table. The following rules can be used as a reference.

In [33]:
from dictionary import Dictionary
import nltk
import re

dict_info = Dictionary("dictionary.ini")
table_prop_pattern = dict_info.table_prop_pattern
table_prop_pattern

{'solidus': ['Tsolidus', 'Solidus', 'solidus'],
 'solvus': ['γ′-solvus',
  'γ′Solvus',
  'solvus',
  'Solvus',
  'γ′-transus',
  'Tγ′',
  'solvus-temperature',
  '-solvus'],
 'density': ['density', 'Density', 'densities', 'densityρ', 'pO-'],
 'liquidus': ['liquidus', 'Liquidus', 'Tlq', 'liquidous', 'liquidusus']}

### 4.2 table_ratio_pattern

The rule is used for recognizing the unit of elements in the table. The format of the table_ratio_pattern rule can be represented as 'A|B|...', and the specific rule of A or B is made to match the unit of elements in different writing types. The following rules can be used as a reference.

In [34]:
from dictionary import Dictionary
import nltk
import re

dict_info = Dictionary("dictionary.ini")
table_ratio_pattern = dict_info.table_ratio_pattern
table_ratio_pattern

'[A-Za-z]+\\.?.?\\%|weight\\.?.?\\%|atom\\.?.?\\%|[Aa]t\\.?.?[Pp]ct|[Ww]t\\.?.?[Pp]ct|[Aa]tomic\\.?.?[Pp]ercent|[Ww]eight\\.?.?[Pp]ercent'

### 4.3 table_e_pattern

The rule is used for recognizing the element name in table. The format of table_e_pattern rule is the same as table_ratio_pattern, and the specific rule of A or B is made according to the specific element your material contains. The following rules can be used as a reference.

In [35]:
from dictionary import Dictionary

dict_info = Dictionary("dictionary.ini")
table_e_pattern = dict_info.table_e_pattern
table_e_pattern

'^[A-Z][a-z]?$|[Cc]hromium|[Nn]ickel|[Mm]olybdenum|[Vv]anadium|[Aa]luminum|[Tt]itanium|[Bb]oron|[Cc]arbon|[Ii]ron|[Mm]anganese|[Ss]ilicon|[Pp]hosphorus|[Ss]ulfur|[Tt]ungsten|[Gg]ermanium|[Rr]henium|[Nn]iobium|[Ii]ron|[Pp]ickaxe|[Hh]afnium|[Rr]uthenium|[Ii]ridium|[Yy]ttrium|[Pp]hosphorus|[Ss]ulphur|[Pp]latinum'

### 4.4 ele_list

The dictionary contains all abbreviations of alloy element, it is the set of all values in rule ele_to_abr (3.2).

In [36]:
from dictionary import Dictionary

dict_info = Dictionary("dictionary.ini")
ele_list = dict_info.ele_list
ele_list

['Co',
 'Al',
 'W',
 'Ni',
 'Ti',
 'Cr',
 'Ge',
 'Ta',
 'B',
 'Mo',
 'Re',
 'Nb',
 'Mn',
 'Si',
 'V',
 'Fe',
 'Zr',
 'Hf',
 'Ru',
 'Ir',
 'La',
 'Y',
 'Mg',
 'C']

### 4.5 unit_pattern_table

The rule is used for recognizing the unit of property in table. The format of unit_pattern_table rule can be represented as {'prop_name':'A|B|..',..}. The A or B rule is made to match the unit of the corresponding property. The following rules can be used as a reference.

In [37]:
from dictionary import Dictionary

dict_info = Dictionary("dictionary.ini")
unit_pattern_table = dict_info.unit_pattern_table
prop_name = 'solvus'
print(unit_pattern_table)
unit_pattern_table[prop_name]

{'solvus': '°C|degC|\\(K\\)', 'density': '[Kk]g\\/m3?|g\\/cm3?|gm\\/cm3?|g\\s+cm\\-?3?|kgm\\s+\\-?3?|[Kk]g\\s+m\\-?3?|[Kk]g\\*m\\-?3?|g\\*cm\\-?3?|gcm\\-?3?|J\\/mm3?|g\\/cc|A\\/dm2?|mA\\/cm2?|g\\/mL|mAcm\\-?2?|[Mm]g\\/m3?', 'solidus': '°C|degC|\\(K\\)', 'liquidus': '°C|degC|\\(K\\)'}


'°C|degC|\\(K\\)'

### 4.6 table_number_pattern

The rule is used for recognizing the parameter of property in the table. The format of unit_pattern_table rule is same as unit_pattern_table rule. The following rules can be used as a reference.

In [38]:
from dictionary import Dictionary

dict_info = Dictionary("dictionary.ini")
table_number_pattern = dict_info.table_number_pattern
prop_name = 'solvus'
print(table_number_pattern)
table_number_pattern[prop_name]

{'solvus': '^\\D{0,1}[0-9]{3,4}', 'density': '^\\W{0,1}\\d+', 'solidus': '^\\D{0,1}[0-9]{3,4}', 'liquidus': '^\\D{0,1}[0-9]{3,4}'}


'^\\D{0,1}[0-9]{3,4}'

### 4.7 table_prop_pattern_words

The function and format of the rule are similar to table_prop_pattern. Differences between them are that under this rule when the table cell contains both string A, string B, and string C, the cell content will be used as the property name. Under the table_prop_pattern rule, as long as either string A, string B, or string C exists in the cell, the cell content is used as the property name. The rule is used for recognizing the parameter of property in the table. The format of unit_pattern_table rule is the same as unit_pattern_table rule. The following rules can be used as a reference.

In [39]:
from dictionary import Dictionary

dict_info = Dictionary("dictionary.ini")
table_prop_pattern_words = dict_info.table_prop_pattern_words
prop_name = 'solvus'
print(table_prop_pattern_words)
table_prop_pattern_words[prop_name]

{'solvus': ['T', 'gamma', 'prime'], 'solidus': ['solidus'], 'liquidus': ['liquidus'], 'density': ['density']}


['T', 'gamma', 'prime']

As we can see, only when a table cell contains both string 'T', string 'gamma', and string 'prime', the cell content will be viewed as the property name.

### 4.8 table_units

The content of table_units is used for table parsing and is referred to "Table extractor" (https://github.com/olivettigroup/table_extractor).